# CodeSearchNet Data Source Notice

In [ ]:
!mkdir /content/CodeSearchNet

mkdir: cannot create directory ‘/content/CodeSearchNet’: File exists


In [ ]:
%%capture
!pip install docopt

After the CodeSearchNet dataset was archieved, the S3 bucket was taken offline. As a result, following the installation on the github installation guide will not work. A short illustration can be seen down below.

In [ ]:
import os
from subprocess import call, check_call, CalledProcessError

destination_dir = "/content/CodeSearchNet"

if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
os.chdir(destination_dir)

try:
    language = "python"
    check_call(['wget', f'https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/{language}.zip', '-O', f'{language}.zip'])
    check_call(['unzip', f'{language}.zip'])
    check_call(['rm', f'{language}.zip'])
except CalledProcessError as e:
    print(f"Error: {e}")
    print(f"Error executing command {e.cmd}")
    print(f"Returned code {e.returncode}")

Error: Command '['wget', 'https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip', '-O', 'python.zip']' returned non-zero exit status 8.
Error executing command ['wget', 'https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip', '-O', 'python.zip']
Returned code 8


Instead, we download the dataset from Hugging Face. Updating `datasets` might not be necessary but might sometimes be helpful to avoid errors concering caching in the local file system

# Data Fetching

In [1]:
%%capture

%pip install -U datasets

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("code_search_net", "python")

train_data = dataset["train"]
test_data = dataset["test"]
validation_data = dataset["validation"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

code_search_net.py: 0.00B [00:00, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

We can inspect the contents of the dataset object for the training, testing, and validation datasets.

In [3]:
print(train_data.features.keys())

dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])


# Question Generation Pipeline

In [4]:
import itertools
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer
)
from nltk import sent_tokenize

from typing import (
    Tuple,
    Dict,
    Literal,
    List,
    Any,
    Generator,
    overload,
    Union
)

class QGPipeline:
    def __init__(
        self,
        model: str,
        qg_format: Literal["highlight"] = "highlight",
        exclude_after: List[str] = [],
        use_cuda: bool = False
    ):

        self.model = AutoModelForSeq2SeqLM.from_pretrained(model)
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.qg_format = qg_format

        assert self.model.__class__.__name__ == "T5ForConditionalGeneration"

        self.device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")
        self.model.to(self.device)
        self.model.eval()
        self.use_cuda = use_cuda
        self._exclude_after = exclude_after
        print(f"Using {self.device}")

    def __call__(self, input: Union[Tuple[str, str], List[Tuple[str, str]]]):
        if isinstance(input, tuple):
            # Handle single input
            func_name, docstring = input
            questions = self._generate_questions(func_name, docstring)
            output = [{'answer': func_name, 'question': que} for que in questions]
            if output:
                 return output[0]
            else:
                 return {}

        elif isinstance(input, list) and all(isinstance(item, tuple) for item in input):
            # Handle batch input with proper error handling
            return self._process_batch_generator(input)
        else:
            raise TypeError("Invalid input type. Expected a tuple (func_name, docstring) or a list of such tuples.")


    def _process_batch_generator(self, batch_input: List[Tuple[str, str]]) -> Generator[Dict[str, Any], None, None]:
        """
        Process batch input and yield results with error handling per item
        """
        for i, (func_name, docstring) in enumerate(batch_input):
            try:
                questions = self._generate_questions(func_name, docstring)
                output = [{'answer': func_name, 'question': que} for que in questions]

                if output:
                    yield {
                        'success': True,
                        'index': i,
                        'function_name': func_name,
                        'docstring': docstring,
                        'model_output': output[0],
                        'error': None
                    }
                else:
                    yield {
                        'success': False,
                        'index': i,
                        'function_name': func_name,
                        'docstring': docstring,
                        'model_output': {},
                        'error': 'No questions generated'
                    }

            except Exception as e:
                yield {
                    'success': False,
                    'index': i,
                    'function_name': func_name,
                    'docstring': docstring,
                    'model_output': {},
                    'error': str(e)
                }

    def _generate_questions(self, func_name, docstring):
        #TODO: This can be re-written in a more forceful way for the llm
        inputs = self._prepare_inputs_for_question_extraction(func_name, docstring)

        inputs = self._tokenize(inputs, padding=True, truncation=True)

        with torch.no_grad():
            outs = self.model.generate(
                input_ids=inputs['input_ids'].to(self.device),
                attention_mask=inputs['attention_mask'].to(self.device),
                num_beams=4,

                max_length=32
            )

        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]

        return questions

    def _tokenize(self, inputs, padding=True, truncation=True, add_special_tokens=True, max_length=512):
        tokenized_inputs = self.tokenizer(
            inputs,
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding=padding,
            return_tensors="pt"
        )
        return tokenized_inputs

    def _prepare_inputs_for_question_extraction(self, func_name, docstring):
        #NOTE: experimental, consider removing :params and :return values
        #manual observation suggests the model struggles to understand the pupose of the function in their presense
        for string in self._exclude_after:
            param_idx = docstring.find(string)
            if param_idx != -1:
                docstring = docstring[:param_idx]
            docstring = docstring.strip()
        input = f"answer: <hl>The function is {func_name}<hl>. Context: {docstring} </s>"

        return [input]

    @property
    def exclude_after(self):
        return self._exclude_after

    @exclude_after.setter
    def exclude_after(self, value):
        self._exclude_after = value

In [5]:
finetuned_t5 = QGPipeline(model="valhalla/t5-base-qg-hl")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Using cpu


In [9]:
func_name = train_data[4]['func_name']
docstring = train_data[4]['func_documentation_string']
print(docstring)
finetuned_t5((func_name, docstring))

Read a single frame from the connection.


{'answer': 'WebSocketCommonProtocol.read_frame',
 'question': 'What is the function that reads a single frame from a connection?'}

The result looks promising. Let's run the model for the first 20 doc strings in our dataset

In [ ]:
for i in range(10):
    func_name = train_data[i]['func_name']
    p = docstring = train_data[i]['func_documentation_string']
    print(f"========Sample{i+1}==========")
    idx = docstring.find(":param")
    if idx != -1:
        p = docstring[:idx]
    p = docstring[:].strip()
    print(f"Docstring: {docstring}")
    print(finetuned_t5(func_name, docstring))


========Sample1==========
Docstring: Display slug with level by language.
[{'answer': 'show_slug_with_level', 'question': 'What is the name of the function that displays a slug with level by language?'}]
========Sample2==========
Docstring: Render the last 10 revisions of a page content with a list using
        the ``pages/revisions.html`` template
[{'answer': 'show_revisions', 'question': 'What is the name of the function that shows the last 10 revisions of a page?'}]
========Sample3==========
Docstring: Method that parse the imageplaceholder template tag.
[{'answer': 'do_videoplaceholder', 'question': 'What is the name of the method that parses the imageplaceholder template tag?'}]
========Sample4==========
Docstring: Return Pages with given tag

    Syntax::

        {% get_pages_with_tag <tag name> as <varname> %}

    Example use:
        {% get_pages_with_tag "footer" as pages %}
[{'answer': 'do_get_pages_with_tag', 'question': 'What is the name of the function that returns page

In [18]:
zipped5 = zip(train_data[:5]['func_name'], train_data[:5]['func_documentation_string'])

generator = finetuned_t5(list(zipped5))
print(generator)

for item in generator:
    ans, ques = item['function_name'], item['model_output']['question']
    print(f"========Sample==========")
    print(f"Question: {ques}")
    print(f"Answer: {ans}")
    print()

<generator object QGPipeline._process_batch_generator at 0x7833286165c0>
========Sample==========
Question: What is the function that ensures that the WebSocket connection is open?
Answer: WebSocketCommonProtocol.ensure_open

========Sample==========
Question: What is the function that reads incoming messages and puts them in a queue?
Answer: WebSocketCommonProtocol.transfer_data

========Sample==========
Question: What is the function that reads a single message from the connection?
Answer: WebSocketCommonProtocol.read_message

========Sample==========
Question: What is the function that reads a single data frame from a connection?
Answer: WebSocketCommonProtocol.read_data_frame

========Sample==========
Question: What is the function that reads a single frame from a connection?
Answer: WebSocketCommonProtocol.read_frame



# Dataset Processor

In [13]:
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import time
from typing import List, Dict, Any
from datasets import Dataset, DatasetDict

class DocstringDatasetProcessor:
    def __init__(self,
                 hf_dataset_name: str,
                 batch_size: int = 1000,
                 token: str = None,
                 save_locally: bool = False,
                 local_cache_dir: str = "./cache",
                 private_repo: bool = False):

        self.hf_dataset_name = hf_dataset_name
        self.batch_size = batch_size
        self.private_repo = private_repo
        self.local_cache_dir = Path(local_cache_dir)
        self.local_cache_dir.mkdir(exist_ok=True)
        self.save_locally = save_locally

        self.processed_count = 0
        self.failed_count = 0
        self.all_generated_data = []

        self.token = token

    def process_batch(self, batch_data: List[Tuple[str, str]], pipeline: QGPipeline, batch_id: int) -> List[Dict[str, Any]]:
        """Process a batch of (func_name, docstring) tuples with individual error handling"""
        batch_results = []
        batch_success_count = 0
        batch_failure_count = 0

        print(batch_data)

        # Process the entire batch through pipeline
        try:
            generator = pipeline(batch_data)
            for result in generator:

                if result['success']:
                    batch_results.append({
                        'function_name': result['function_name'],
                        'docstring': result['docstring'],
                        'question': result['model_output']['question'],
                    })
                    batch_success_count += 1
                else:
                    print(
                        f"Failed to process {result['function_name']}: {result['error']}"
                    )
                    batch_failure_count += 1

        except Exception as e:
            # Catastrophic failure - entire batch failed
            print(f"Catastrophic batch failure {batch_id}: {e}")
            batch_failure_count = len(batch_data)
            batch_success_count = 0

        self.processed_count += batch_success_count
        self.failed_count += batch_failure_count

        print(
            f"Batch {batch_id}: {batch_success_count} successful, "
            f"{batch_failure_count} failed out of {len(batch_data)} items"
        )

        if self.save_locally and batch_results:
            self._save_batch_locally(batch_results, batch_id)

        return batch_results

    def _save_batch_locally(self, batch_results: List[Dict], batch_id: int):
        batch_file = self.local_cache_dir / f"batch_{batch_id}.jsonl"
        with open(batch_file, 'w') as f:
            for item in batch_results:
                json.dump(item, f)
                f.write('\n')

    def process_full_dataset(self, dataset, pipeline, start_idx: int = 0):
        """Process the entire data set and upload to hugging face"""

        print(f"Starting processing of {len(dataset)} items from index {start_idx}")
        start_time = time.time()

        for batch_start in tqdm(range(start_idx, len(dataset), self.batch_size),
                                desc="Processing batches"):
            batch_end = min(batch_start + self.batch_size, len(dataset))
            batch_data = dataset[batch_start:batch_end]
            batch_id = batch_start // self.batch_size

            batch_results = self.process_batch(batch_data, pipeline, batch_id)
            self.all_generated_data.extend(batch_results)

            # print progress
            if batch_id % 10 == 0:
                elapsed = time.time() - start_time
                rate = self.processed_count / elapsed if elapsed > 0 else 0
                print(f"Processed {self.processed_count} items in {elapsed:.2f} seconds. Rate: {rate:.2f} items/sec")

            #final statistics
            total_time = time.time() - start_time
            print(f"Processed {self.processed_count} items in {total_time:.2f} seconds. Rate: {self.processed_count / total_time:.2f} items/sec")

            self._upload_to_hf()

    def _upload_to_hf(self):
        if self.token is None:
            raise ValueError("Hugging Face token not provided")
        try:
            print("Creating Hugging Face dataset")

            dataset = Dataset.from_list(self.all_generated_data)

            dataset_dict = DatasetDict({
                'train': dataset
            })

            #notice this might introduce unneccessary inefficiencies
            dataset_dict = dataset_dict.map(
                lambda x: {
                    **x,
                    'id': f"{x['function_name']}_{hash(x['docstring']) % 10000}"
                }
            )

            print(f"Uploading dataset to {self.hf_dataset_name}...")

            dataset_dict.push_to_hub(
                self.hf_dataset_name,
                token=self.token,
                private=self.private_repo,
                commit_message=f"Add {len(self.all_generated_data)} docstring-question pairs"
            )

            print(f"Successfully uploaded dataset to https://huggingface.co/datasets/{self.hf_dataset_name}")

        except Exception as e:
            print(f"Error uploading to Hugging Face: {e}")
            if self.save_locally:
                print("Data is available locally in cached directory")
            raise

    def load_from_hf(self):
        """Load the dataset from Hugging Face"""

        from datasets import load_dataset

        try:
            dataset = load_dataset(self.hf_dataset_name, token=self.token)
            print(f"Successfully loaded dataset from {self.hf_dataset_name}")
            return dataset
        except Exception as e:
            print(f"Error loading dataset from Hugging Face: {e}")
            raise

    #TODO: resume processing from local cache file
    #TODO: upload from colab cache to permanent file location (local or drive)

In [7]:
# Constants

from google.colab import userdata

HUGGING_FACE_TOKEN = userdata.get("HUGGING_FACE_TOKEN")
hf_dataset_name = "mrinjera/testing"

In [14]:
dataset_processor = DocstringDatasetProcessor(hf_dataset_name, batch_size=10, token=HUGGING_FACE_TOKEN)

batch_raw_data = zip(train_data[:200]['func_name'], train_data[:200]['func_documentation_string'])
batch_zip_data = list(batch_raw_data)

pipeline = finetuned_t5
dataset_processor.process_full_dataset(batch_zip_data, pipeline)

Starting processing of 200 items from index 0


Processing batches:   0%|          | 0/20 [00:00<?, ?it/s]

[('WebSocketCommonProtocol.ensure_open', "Check that the WebSocket connection is open.\n\n        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't."), ('WebSocketCommonProtocol.transfer_data', 'Read incoming messages and put them in a queue.\n\n        This coroutine runs in a task until the closing handshake is started.'), ('WebSocketCommonProtocol.read_message', 'Read a single message from the connection.\n\n        Re-assemble data frames if the message is fragmented.\n\n        Return ``None`` when the closing handshake is started.'), ('WebSocketCommonProtocol.read_data_frame', 'Read a single data frame from the connection.\n\n        Process control frames received before the next data frame.\n\n        Return ``None`` if a close frame is encountered before any data frame.'), ('WebSocketCommonProtocol.read_frame', 'Read a single frame from the connection.'), ('WebSocketCommonProtocol.write_close_frame', 'Write a close frame if and only if the connection state is OP

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Uploading dataset to mrinjera/testing...


...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/37.9k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/382 [00:00<?, ?B/s]

Processing batches: 100%|██████████| 20/20 [09:42<00:00, 29.13s/it]

Successfully uploaded dataset to https://huggingface.co/datasets/mrinjera/testing


In [44]:
for i in range(10):
    print(train_data[i]['func_documentation_string'])

Check that the WebSocket connection is open.

        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't.
Read incoming messages and put them in a queue.

        This coroutine runs in a task until the closing handshake is started.
Read a single message from the connection.

        Re-assemble data frames if the message is fragmented.

        Return ``None`` when the closing handshake is started.
Read a single data frame from the connection.

        Process control frames received before the next data frame.

        Return ``None`` if a close frame is encountered before any data frame.
Read a single frame from the connection.
Write a close frame if and only if the connection state is OPEN.

        This dedicated coroutine must be used for writing close frames to
        ensure that at most one close frame is sent on a given connection.
Send a Ping frame and wait for a Pong frame at regular intervals.

        This coroutine exits when the connection terminates and o

# Model Training

In [18]:
import os
import torch
import pandas as pd
from datasets import Dataset, load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from typing import List, Dict, Tuple
from google.colab import userdata


DATASET_NAME = "mrinjera/testing"
HF_TOKEN = userdata.get("HUGGING_FACE_TOKEN")
MODEL_NAME = "microsoft/codebert-base"
OUTPUT_MODEL_PATH = "./sbert-function-retrieval"
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-5
WARMUP_STEPS = 1000
EVALUATION_STEPS = 5000

In [26]:
def load_and_process_dataset(dataset_name: str, token: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load the dataset and process it for training."""
    print(f"Loading dataset: {dataset_name}")

    dataset = load_dataset(dataset_name, token=token)

    #TODO: for clarity, have a bool param that controls the behavior
    if 'train' not in dataset or 'test' not in dataset:
        dataset = dataset['train'].shuffle(seed=42)
        train_size = int(0.8 * len(dataset))
        train_df = dataset.select(range(train_size)).to_pandas()
        test_df = dataset.select(range(train_size, len(dataset))).to_pandas()
    else:
        train_df = dataset['train'].to_pandas()
        test_df  = dataset['test'].to_pandas()

    print(f"Train dataset loaded with {len(train_df)} examples")
    print(f"Test dataset loaded with {len(test_df)} examples")
    print(f"Dataset columns: {train_df.columns.tolist()}")

    required_columns = ['function_name', 'docstring', 'question', 'id']
    for df_name, df in [('train', train_df), ('test', test_df)]:
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns in {df_name} split: {missing_columns}")

    train_df = train_df.dropna(subset=['docstring', 'question'])
    test_df = test_df.dropna(subset=['docstring', 'question'])

    print(f"After removing missing values:")
    print(f"  Train: {len(train_df)} examples")
    print(f"  Test: {len(test_df)} examples")

    return train_df, test_df

In [28]:
def create_training_examples(df: pd.DataFrame) -> List[InputExample]:
    """Create InputExample objects for SBERT training."""
    examples = []

    for idx, row in df.iterrows():
        # Create input example with question as query and docstring as positive document
        example = InputExample(
            texts=[row['question'], row['docstring']],
            label=1.0  # Positive pair
        )
        examples.append(example)

    print(f"Created {len(examples)} training examples")
    return examples

def create_evaluation_data(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], Dict[str, set]]:
    """Create evaluation data for Information Retrieval evaluation."""
    # Split data for evaluation
    eval_df = df.sample(n=min(1000, len(df) // 10), random_state=42)

    queries = {}
    corpus = {}
    relevant_docs = {}

    for idx, row in eval_df.iterrows():
        query_id = f"q_{idx}"
        doc_id = f"d_{idx}"

        queries[query_id] = row['question']
        corpus[doc_id] = row['docstring']
        relevant_docs[query_id] = {doc_id}

    print(f"Created evaluation data with {len(queries)} queries and {len(corpus)} documents")
    return queries, corpus, relevant_docs

def create_validation_split(train_df: pd.DataFrame,
                          validation_size: float = 0.1) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Create a validation split from training data for monitoring during training."""
    train_split, val_split = train_test_split(
        train_df,
        test_size=validation_size,
        random_state=42,
        stratify=None
    )

    print(f"Created validation split:")
    print(f"  Training: {len(train_split)} examples")
    print(f"  Validation: {len(val_split)} examples")

    return train_split, val_split

In [21]:
def initialize_model(model_name: str) -> SentenceTransformer:
    """Initialize the SBERT model."""
    print(f"Initializing model: {model_name}")

    model = SentenceTransformer(model_name)

    print(f"Model max sequence length: {model.max_seq_length}")
    print(f"Model device: {model.device}")

    return model

def train_model(model: SentenceTransformer,
                train_examples: List[InputExample],
                val_queries: Dict[str, str],
                val_corpus: Dict[str, str],
                val_relevant_docs: Dict[str, set],
                device: torch.device) -> SentenceTransformer:
    """Train the SBERT model with Multiple Negatives Ranking Loss."""

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)

    train_loss = losses.MultipleNegativesRankingLoss(model)

    val_evaluator = InformationRetrievalEvaluator(
        queries=val_queries,
        corpus=val_corpus,
        relevant_docs=val_relevant_docs,
        name="validation-eval"
    )

    # calculate warmup steps
    num_train_steps = len(train_dataloader) * EPOCHS
    warmup_steps = min(WARMUP_STEPS, num_train_steps // 10)

    print(f"Training configuration:")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Epochs: {EPOCHS}")
    print(f"  Learning rate: {LEARNING_RATE}")
    print(f"  Warmup steps: {warmup_steps}")
    print(f"  Total training steps: {num_train_steps}")
    print(f"  Evaluation steps: {EVALUATION_STEPS}")

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=val_evaluator,
        epochs=EPOCHS,
        evaluation_steps=EVALUATION_STEPS,
        warmup_steps=warmup_steps,
        output_path=OUTPUT_MODEL_PATH,
        optimizer_params={'lr': LEARNING_RATE},
        save_best_model=True,
        show_progress_bar=True
    )

    return model

In [22]:
def evaluate_on_test_set(model: SentenceTransformer,
                        test_queries: Dict[str, str],
                        test_corpus: Dict[str, str],
                        test_relevant_docs: Dict[str, set]) -> Dict[str, float]:
    """Evaluate the trained model on the test set."""
    print("Evaluating model on test set...")

    # Create test evaluator
    test_evaluator = InformationRetrievalEvaluator(
        queries=test_queries,
        corpus=test_corpus,
        relevant_docs=test_relevant_docs,
        name="test-eval"
    )

    # Evaluate
    test_score = test_evaluator(model, output_path=OUTPUT_MODEL_PATH)

    print(f"Test evaluation completed. Score: {test_score}")
    return test_score

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_df, test_df = load_and_process_dataset(DATASET_NAME, HF_TOKEN)

train_split_df, val_split_df = create_validation_split(train_df)

train_examples = create_training_examples(train_split_df)

val_queries, val_corpus, val_relevant_docs = create_evaluation_data(val_split_df)

test_queries, test_corpus, test_relevant_docs = create_evaluation_data(test_df)


# model = initialize_model(MODEL_NAME)

# trained_model = train_model(
#     model=model,
#     train_examples=train_examples,
#     val_queries=val_queries,
#     val_corpus=val_corpus,
#     val_relevant_docs=val_relevant_docs,
#     device=device
# )

# # Load best model for final evaluation
# best_model = SentenceTransformer(OUTPUT_MODEL_PATH)

# # Evaluate on test set
# test_score = evaluate_on_test_set(
#     model=best_model,
#     test_queries=test_queries,
#     test_corpus=test_corpus,
#     test_relevant_docs=test_relevant_docs
# )


Loading dataset: mrinjera/testing
Train dataset loaded with 160 examples
Test dataset loaded with 40 examples
Dataset columns: ['function_name', 'docstring', 'question', 'id']
After removing missing values:
  Train: 160 examples
  Test: 40 examples
Created validation split:
  Training: 144 examples
  Validation: 16 examples
Created 144 training examples
Created evaluation data with 1 queries and 1 documents
Created evaluation data with 4 queries and 4 documents
